In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [2]:
from ktrain import text

In [174]:
merged_aspirational_df = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_Aspirational_sentence_level_batch_1_jaccard.csv", encoding='utf-8')
merged_aspirational_df

training_df, test_df = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [177]:
merged_aspirational_df

,sentence,label,phrase
0,why am i here?,0,"[""Ever since I was little I wanted to be a doc..."
1,well why does anyone pursue a higher education?,0,"[""Ever since I was little I wanted to be a doc..."
2,to better one self and be able to succeed late...,0,"[""Ever since I was little I wanted to be a doc..."
3,ever since i was little i wanted to be a docto...,1,"[""Ever since I was little I wanted to be a doc..."
4,i always wanted to be able to help people and ...,0,"[""Ever since I was little I wanted to be a doc..."
...,...,...,...
4699,"i want to better myself, improve myself in eve...",0,['I want to be able to apply my knowledge to m...
4700,i have no problems applying myself when it com...,0,['I want to be able to apply my knowledge to m...
4701,grow mentally so that i can apply more physica...,0,['I want to be able to apply my knowledge to m...
4702,use what i learn and apply it in whatever care...,0,['I want to be able to apply my knowledge to m...


In [127]:
training_df.shape

(3763, 3)

In [128]:
training_df.head()

,sentence,label,phrase
1341,one is obviously this class is a requirement f...,0,['I am here so that I can enhance my education...
3597,to get a better and broader perspective of life.,1,['I am here because I wanted a significant cha...
389,that is what i am in school for but it is not ...,0,['I would like to be a cardiothoracic surgeon.']
1798,"i should have dropped the class, it was a very...",0,['This semester I want to prove to myself that...
1827,i was apart of a sci course last semester and ...,0,['I am here because I am dedicated to my succe...


In [6]:
test_df = pd.read_csv('../../data/csv/attainment_sentence_level_test_data.csv',encoding='utf-8')

In [176]:
test_df["label"]

447     1
727     0
2681    0
1538    0
2807    0
       ..
1017    0
4666    0
2437    0
2022    0
1950    0
Name: label, Length: 941, dtype: int64

In [130]:
test_df.head()

,sentence,label,phrase
447,also i want to be an optometrist and this clas...,1,['Also I want to be an optometrist and this cl...
727,unfortunately it is a requirement for biology ...,0,"[""I am here because this class is one of many ..."
2681,i am here to learn beyond my prior knowledge.,0,"[""I'm here because I need this class to gradua..."
1538,i have no specific direction in my life other ...,0,['I want to try to get into veterinary school ...
2807,im here because i want to pursue a career in t...,0,"['More specifically, this class will get me on..."


In [131]:
MAXLEN = 150

In [132]:
X = training_df['sentence']
y = training_df['label']

In [133]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [134]:
# Convert to lists of strings if not already in that format
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [135]:
model_name = 'distilbert-base-uncased'

In [136]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
# Get the classifier model without specifying optimizer
distillbert_base_model = distillbert_transformer.get_classifier()

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 39
	99percentile : 54


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 20
	95percentile : 39
	99percentile : 56


In [137]:
import tensorflow as tf

In [138]:
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)

In [139]:
print(distillbert_learner.get_weight_decay())

None


In [160]:
distillbert_learner.set_weight_decay(0.001)

In [161]:
# Train the model
distillbert_learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
502/502 [==============================] - 200s 358ms/step - loss: 0.1347 - accuracy: 0.9399 - val_loss: 0.4047 - val_accuracy: 0.8513
Epoch 2/5
502/502 [==============================] - 157s 312ms/step - loss: 0.1188 - accuracy: 0.9439 - val_loss: 0.3903 - val_accuracy: 0.8526
Epoch 3/5
502/502 [==============================] - 151s 301ms/step - loss: 0.1126 - accuracy: 0.9508 - val_loss: 0.3739 - val_accuracy: 0.8513
Epoch 4/5
502/502 [==============================] - 173s 345ms/step - loss: 0.1065 - accuracy: 0.9571 - val_loss: 0.4958 - val_accuracy: 0.8606
Epoch 5/5
502/502 [==============================] - 168s 335ms/step - loss: 0.0954 - accuracy: 0.9571 - val_loss: 0.4487 - val_accuracy: 0.8513


In [162]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

24/24 [==============================] - 20s 524ms/step
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       659
           1       0.41      0.44      0.42        94

    accuracy                           0.85       753
   macro avg       0.66      0.67      0.67       753
weighted avg       0.86      0.85      0.85       753



array([[600,  59],
       [ 53,  41]])

In [163]:
distillbert_learner.model.summary()

Model: "tf_distil_bert_for_sequence_classification_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_599 (Dropout)       multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [165]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [166]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [167]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [168]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [169]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 725, False Positive: 98, False Negative: 67, True Positive: 51


In [170]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.88      0.90       823
           1       0.34      0.43      0.38       118

    accuracy                           0.82       941
   macro avg       0.63      0.66      0.64       941
weighted avg       0.84      0.82      0.83       941
 



In [171]:
#distillbert_predictor.save('./model/distilbert_base_uncased_model') # 256 MB

In [172]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.6565634203101509
